In [28]:
import numpy as np
import pandas as pd
import sys 
import os
sys.path.append(os.path.relpath('../'))
from sklearn import metrics
%matplotlib inline

In [29]:
analys_personal_data = pd.read_csv('../../../Data/For_analysis_and_modeling/2nd_quadrimester/analys_personal_data.csv',sep='|')

In [30]:
analys_personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,2,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,3,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,4,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,5,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,6,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [31]:
interval_cols = []

In [32]:
analys_personal_data_model = analys_personal_data.copy()

In [33]:
analys_personal_data_model.drop(['expediente','cod_plan'],axis=1,inplace=True)

In [34]:

note_bcket_array = np.array([5, 6.5, 8, 9.5, 10, 11.5, 13, 14])
analys_personal_data_model['nota_admision_def'] = pd.cut(
    analys_personal_data_model['nota_admision_def'], note_bcket_array, include_lowest=True)
interval_cols.append('nota_admision_def')

year_bcket_array = np.array([18, 20, 25, 30, 35, 60])
analys_personal_data_model['edad_acceso'] = pd.cut(
    analys_personal_data_model['edad_acceso'], year_bcket_array, include_lowest=True)
interval_cols.append('edad_acceso')


distance_bcket_array = np.array([0, 45, 90, 135, 250, 500, 1892])
analys_personal_data_model['distance'] = pd.cut(
    analys_personal_data_model['distance'], distance_bcket_array, include_lowest=True)
interval_cols.append('distance')

note_bcket_array = np.array([0, 1.5, 3, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_median'] = pd.cut(
    analys_personal_data_model['cum_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_median')
note_bcket_array = np.array([0, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_pass_median'] = pd.cut(
    analys_personal_data_model['cum_pass_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_median')
note_bcket_array = np.array([0, 1.5, 3, 5])
analys_personal_data_model['cum_fail_median'] = pd.cut(
    analys_personal_data_model['cum_fail_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_fail_median')

ratio_bcket_array = np.array([0, 0.25, 0.5, 0.75, 1])
analys_personal_data_model['cum_pass_ratio'] = pd.cut(
    analys_personal_data_model['cum_pass_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_ratio')

analys_personal_data_model['cum_absent_ratio'] = pd.cut(
    analys_personal_data_model['cum_absent_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_absent_ratio')

std_bcket_array = np.array([0, 0.05,1.15,1.25,2.15,3.25])
analys_personal_data_model['std_deviation'] = pd.cut(
    analys_personal_data_model['std_deviation'], std_bcket_array, include_lowest=True)
interval_cols.append('std_deviation')


In [35]:
analys_personal_data_model.head()

,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,"(6.5, 8.0]",D,"(20.0, 25.0]","(250.0, 500.0]",S,"(-0.001, 0.25]","(-0.001, 0.25]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(3.0, 5.0]"
1,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Formación Profesional,"(6.5, 8.0]",H,"(30.0, 35.0]","(500.0, 1892.0]",S,"(0.25, 0.5]","(0.5, 0.75]","(1.25, 2.15]","(4.5, 6.0]","(4.5, 6.0]","(1.5, 3.0]"
2,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Formación Profesional,"(6.5, 8.0]",D,"(25.0, 30.0]","(90.0, 135.0]",N,"(-0.001, 0.25]","(0.25, 0.5]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(1.5, 3.0]"
3,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Selectividad,"(6.5, 8.0]",D,"(20.0, 25.0]","(90.0, 135.0]",N,"(0.25, 0.5]","(0.25, 0.5]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(1.5, 3.0]"
4,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,"(6.5, 8.0]",H,"(20.0, 25.0]","(90.0, 135.0]",N,"(0.75, 1.0]","(-0.001, 0.25]","(1.25, 2.15]","(6.0, 7.5]","(6.0, 7.5]","(-0.001, 1.5]"


In [36]:
analys_personal_data_model.to_csv('./analys_personal_data_model.csv',index=False)

In [37]:
analys_personal_data_clust = pd.read_csv('./analys_personal_data_clust.csv',index_col=0)

In [38]:
analys_personal_data_clust.head()

,des_plan GRADO EN EDIFICACIÓN,des_plan GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIVILES,des_plan GRADO EN INGENIERÍA CIVIL - HIDROLOGÍA,des_plan GRADO EN INGENIERÍA CIVIL - TRANSPORTES Y SERVICIOS URBANOS,des_plan GRADO EN INGENIERÍA DE SONIDO E IMAGEN EN TELECOMUNICACIÓN,des_plan GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DE COMPUTADORES,des_plan GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DEL SOFTWARE,anio_apertura_expediente 2008-09,anio_apertura_expediente 2009-10,anio_apertura_expediente 2010-11,...,"cum_pass_median (-0.001, 4.5]","cum_pass_median (4.5, 6.0]","cum_pass_median (6.0, 7.5]","cum_pass_median (7.5, 9.0]","cum_pass_median (9.0, 10.0]","cum_fail_median (-0.001, 1.5]","cum_fail_median (1.5, 3.0]","cum_fail_median (3.0, 5.0]",labels,abandona
1,False,True,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,True,2,0
2,False,True,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,True,False,1,1
3,False,True,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,True,False,1,0
4,False,True,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,True,False,2,1
5,False,True,False,False,False,False,False,False,True,False,...,False,False,True,False,False,True,False,False,2,0


In [39]:
def get_dunn_index(data, labels):
    import dunn_index
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(data)
    return dunn_index.dunn(labels,distances)

In [40]:
metrics.silhouette_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


0.06263218378538227

In [41]:
metrics.calinski_harabasz_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


61.86225097859311

In [42]:
metrics.davies_bouldin_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


3.717861655294941

In [43]:
get_dunn_index(analys_personal_data_clust,analys_personal_data_clust['labels'])

0.3110855084191276

In [44]:
import pandas as pd
def get_impurity(dropout_cl0: pd.core.series.Series, dropout_cl1:pd.core.series.Series):
    impurity = -1
    n = len (dropout_cl0) + len (dropout_cl1)
    if len(dropout_cl0[dropout_cl0 == 0]) > len(dropout_cl0[dropout_cl0 == 1]):
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 0]))/n
    else:
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 1]))/n
    if len(dropout_cl1[dropout_cl1 == 0]) > len(dropout_cl1[dropout_cl1 == 1]):
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 0]))/n)
    else:
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 1]))/n)
        
    return impurity

In [45]:
get_impurity(analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'], analys_personal_data_clust[analys_personal_data_clust['labels']==2]['abandona'])

0.395375070501974

In [46]:
analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'].value_counts(normalize=True)

0    0.786561
1    0.213439
Name: abandona, dtype: float64

In [47]:
analys_personal_data_clust[analys_personal_data_clust['labels']==2]['abandona'].value_counts(normalize=True)

1    0.574342
0    0.425658
Name: abandona, dtype: float64